## Kelompok 5
 
### 12S18002 - Wiranda Siahaan
### 12S18009 - Novita Hutapea
### 12S18041 - Merika Manurung
### 12S18046 - Tiara Situmorang

**Consume twitter steaming data**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
import socket
import json
import numpy as np
import pandas as pd

Insert your credentials

In [ ]:
#Kindly put your credentials received from twitter developer account
consumer_key='2mTmxHwkZramjVDOewE2dybgQ'
consumer_secret='dJ42E93prTAze5BoF4wAIcxYA6gXfu9rGrHr6gguUt85xcHJSi'
access_token ='1517128047019040768-YMZCcFIf6VLbZXNkJsa79sZk70xFkM'
access_secret='Auwa9WPIktjUCrgQmzceZsdMtMA2hbnebSR7QlmYEdrlq'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [ ]:
for tweet in api.search(q='miniso', lan='en'):    
    print(tweet.text)

In [ ]:
tweetDf = pd.DataFrame(columns = ['User', 'User_statuses_count', 
                             'user_followers','fav_count','User_location','Tweets',])

In [ ]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100000, lang='en').items():
        print(i, end='\r')
        tweetDf.loc[i, 'User'] = tweet.user.name
        tweetDf.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        tweetDf.loc[i, 'user_followers'] = tweet.user.followers_count
        tweetDf.loc[i, 'fav_count'] = tweet.favorite_count
        tweetDf.loc[i, 'User_location'] = tweet.user.location
        tweetDf.loc[i, 'Tweets'] = tweet.text
        
        
        tweetDf.to_csv('{}.csv'.format(file_name))
        i+=1
        if i == 200:
            break
        else:
            pass

In [ ]:
stream(data = ['miniso'], file_name = 'miniso_2021')

In [ ]:
tweetDf.head()